# 🎄 The Christmas Tree Disposal Problem

This notebook explores a **social contagion effect** on a real number line.

Imagine a neighborhood where houses are located at every integer point on the real number line. Every household celebrates Christmas and has a Christmas tree. We will simulate how the decision to dispose of a tree propagates through the neighborhood like a wave.

## Setup

We only need `numpy` and `matplotlib`, both of which are available in Pyodide out of the box.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# Make plots look nice
plt.rcParams.update({
    'figure.figsize': (12, 5),
    'font.size': 12,
    'axes.grid': True,
    'grid.alpha': 0.3,
})

print("Libraries loaded successfully!")

---
## 1. Define the Natural Disposal Distribution

Each household $i$ has a "natural" disposal day $T_i^0$ drawn from a distribution $P(T)$ that:
- Starts at $t = 1$ (December 26) with very low probability
- Peaks several days after Christmas
- Has a long tail (some people wait a long time)

We'll use a **shifted log-normal** distribution to model this.

### 📋 Exercise
Experiment with the `mu` and `sigma` parameters below to shape the distribution. What happens when `sigma` is large vs small?

In [ ]:
def sample_natural_disposal(n_houses, mu=2.0, sigma=0.5, rng=None):
    """
    Sample natural disposal days from a shifted log-normal distribution.
    
    Parameters
    ----------
    n_houses : int
        Number of households.
    mu : float
        Mean of the underlying normal distribution (controls peak location).
    sigma : float
        Std dev of the underlying normal distribution (controls spread).
    rng : np.random.Generator, optional
        Random number generator for reproducibility.
    
    Returns
    -------
    np.ndarray
        Array of natural disposal days (float, >= 1).
    """
    if rng is None:
        rng = np.random.default_rng()
    # Shifted log-normal: minimum disposal day is 1 (Dec 26)
    return 1.0 + rng.lognormal(mean=mu, sigma=sigma, size=n_houses)


# --- Visualize the distribution ---
rng = np.random.default_rng(42)
sample = sample_natural_disposal(10_000, mu=2.0, sigma=0.5, rng=rng)

fig, ax = plt.subplots()
ax.hist(sample, bins=60, density=True, color='forestgreen', alpha=0.7, edgecolor='white')
ax.axvline(np.median(sample), color='red', linestyle='--', label=f'Median = {np.median(sample):.1f} days')
ax.set_xlabel('Days after Christmas (t)')
ax.set_ylabel('Density')
ax.set_title('Natural Disposal Day Distribution $T_i^0$')
ax.legend()
plt.tight_layout()
plt.show()

---
## 2. Model the Social Contagion Effect

If a household sees a neighbor's tree already on the curb ($T_{i \pm 1} < T_i^0$), it may hasten their own decision.

We define the **actual** disposal day as:
$$T_i = \min\!\left(T_i^0,\; \min(T_{i-1}, T_{i+1}) + \Delta\right)$$

where $\Delta \ge 0$ is the **reaction delay** — how many days after seeing a neighbor's tree before a household puts theirs out.

- $\Delta = 0$: instant copycat behavior
- $\Delta = 1$: one-day lag
- Large $\Delta$: neighbors have little influence

### 📋 Exercise
Try different values of `reaction_delay` in the simulation below. What value creates the most dramatic wave effect?

In [ ]:
def simulate(n_houses=201, mu=2.0, sigma=0.5, reaction_delay=1.0, max_iter=200, seed=42):
    """
    Run the Christmas tree disposal simulation with social contagion.
    
    Parameters
    ----------
    n_houses : int
        Total number of houses (centered at 0, so range is [-n//2, n//2]).
    mu, sigma : float
        Parameters of the natural disposal distribution.
    reaction_delay : float
        Days after seeing a neighbor's tree before disposing own tree (Delta).
    max_iter : int
        Maximum number of propagation iterations.
    seed : int
        Random seed for reproducibility.
    
    Returns
    -------
    positions : np.ndarray
        House positions on the number line.
    T0 : np.ndarray
        Natural disposal days.
    T : np.ndarray
        Actual disposal days (after contagion).
    """
    rng = np.random.default_rng(seed)
    half = n_houses // 2
    positions = np.arange(-half, half + 1)
    
    # Draw natural disposal days
    T0 = sample_natural_disposal(len(positions), mu=mu, sigma=sigma, rng=rng)
    T = T0.copy()
    
    # Iteratively propagate the contagion until no more changes
    for _ in range(max_iter):
        T_new = T.copy()
        for i in range(len(T)):
            neighbor_min = np.inf
            if i > 0:
                neighbor_min = min(neighbor_min, T[i - 1])
            if i < len(T) - 1:
                neighbor_min = min(neighbor_min, T[i + 1])
            
            # If a neighbor has already disposed, consider hastening
            influenced_time = neighbor_min + reaction_delay
            T_new[i] = min(T0[i], influenced_time)
        
        if np.allclose(T_new, T):
            break
        T = T_new
    
    return positions, T0, T

print("Simulation function defined.")

---
## 3. Run the Simulation

Let's run the simulation and visualize how the disposal wave propagates through the neighborhood.

In [ ]:
# --- Run with default parameters ---
positions, T0, T = simulate(n_houses=201, mu=2.0, sigma=0.5, reaction_delay=1.0, seed=42)

# --- Plot: Before vs After contagion ---
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Before contagion
colors_before = cm.Greens(T0 / T0.max())
axes[0].bar(positions, T0, width=1.0, color=colors_before, edgecolor='none')
axes[0].set_xlabel('House Position')
axes[0].set_ylabel('Disposal Day')
axes[0].set_title('Natural Disposal Days $T_i^0$ (No Contagion)')

# After contagion
colors_after = cm.Reds(T / T0.max())
axes[1].bar(positions, T, width=1.0, color=colors_after, edgecolor='none')
axes[1].set_xlabel('House Position')
axes[1].set_ylabel('Disposal Day')
axes[1].set_title('Actual Disposal Days $T_i$ (With Contagion, $\\Delta=1$)')

for ax in axes:
    ax.set_xlim(positions[0] - 1, positions[-1] + 1)

plt.tight_layout()
plt.show()

print(f"Average natural disposal day:  {T0.mean():.1f}")
print(f"Average actual disposal day:   {T.mean():.1f}")
print(f"Days saved by contagion:       {T0.mean() - T.mean():.1f}")

---
## 4. Analyze the Results

Let's dig deeper into the contagion effect.

In [ ]:
# --- How much earlier did each house dispose? ---
shift = T0 - T

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Scatter of shift per position
axes[0].scatter(positions, shift, s=6, c=shift, cmap='coolwarm', alpha=0.8)
axes[0].axhline(0, color='gray', linewidth=0.5)
axes[0].set_xlabel('House Position')
axes[0].set_ylabel('Days Earlier (shift)')
axes[0].set_title('Social Contagion Shift per Household')

# Distribution of shifts
axes[1].hist(shift, bins=40, color='steelblue', alpha=0.7, edgecolor='white')
axes[1].set_xlabel('Days Earlier')
axes[1].set_ylabel('Count')
axes[1].set_title('Distribution of Days Saved by Contagion')

plt.tight_layout()
plt.show()

pct_affected = (shift > 0.5).mean() * 100
print(f"{pct_affected:.1f}% of households disposed earlier due to social contagion.")

### 4.1 Effect of Reaction Delay $\Delta$

How does the reaction delay change the contagion dynamics?

In [ ]:
delays = [0.0, 0.5, 1.0, 2.0, 5.0, 10.0]
avg_shifts = []
pct_affected_list = []

for d in delays:
    _, T0_d, T_d = simulate(n_houses=201, reaction_delay=d, seed=42)
    shift_d = T0_d - T_d
    avg_shifts.append(shift_d.mean())
    pct_affected_list.append((shift_d > 0.5).mean() * 100)

fig, ax1 = plt.subplots(figsize=(10, 5))
color1 = 'tab:blue'
ax1.plot(delays, avg_shifts, 'o-', color=color1, linewidth=2, markersize=8)
ax1.set_xlabel('Reaction Delay $\\Delta$ (days)')
ax1.set_ylabel('Avg Days Saved', color=color1)
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()
color2 = 'tab:red'
ax2.plot(delays, pct_affected_list, 's--', color=color2, linewidth=2, markersize=8)
ax2.set_ylabel('% Households Affected', color=color2)
ax2.tick_params(axis='y', labelcolor=color2)

ax1.set_title('Impact of Reaction Delay on Social Contagion')
fig.tight_layout()
plt.show()

### 4.2 Disposal Clustering

Are there clusters of houses that all dispose on similar days?

In [ ]:
# Round disposal days to integers and count how many dispose each day
positions, T0, T = simulate(n_houses=201, reaction_delay=1.0, seed=42)

disposal_days_natural = np.round(T0).astype(int)
disposal_days_actual = np.round(T).astype(int)

max_day = max(disposal_days_natural.max(), disposal_days_actual.max())
days = np.arange(1, max_day + 1)

counts_natural = np.array([(disposal_days_natural == d).sum() for d in days])
counts_actual = np.array([(disposal_days_actual == d).sum() for d in days])

fig, ax = plt.subplots(figsize=(12, 5))
width = 0.35
ax.bar(days - width/2, counts_natural, width, label='Without Contagion', color='forestgreen', alpha=0.7)
ax.bar(days + width/2, counts_actual, width, label='With Contagion', color='crimson', alpha=0.7)
ax.set_xlabel('Disposal Day')
ax.set_ylabel('Number of Households')
ax.set_title('Disposal Day Clustering: Natural vs Social Contagion')
ax.legend()
ax.set_xlim(0, min(30, max_day + 1))
plt.tight_layout()
plt.show()

---
## 5. Your Turn! 🚀

Try modifying the simulation to explore these questions:

1. **What if the reaction delay is random?** Replace the fixed $\Delta$ with a per-household random variable.
2. **What about a larger neighborhood?** Try `n_houses=1001`.
3. **What if only *one* early disposer exists?** Force exactly one house to dispose at $t=1$ and see how the wave propagates.
4. **Different influence rules:** Instead of `min(T0, neighbor_min + Δ)`, try an averaging rule or a probabilistic one.

Use the cells below to experiment!

In [ ]:
# Your experiments here!
